In [1]:
import fastparquet
import pandas as pd

## Data

In [2]:
file_january = 'yellow_tripdata_2023-01.parquet'
file_february = 'yellow_tripdata_2023-02.parquet'

df_jan = fastparquet.ParquetFile(file_january).to_pandas()
df_feb = fastparquet.ParquetFile(file_february).to_pandas()


df = pd.concat([df_jan, df_feb], axis=1)

In [3]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q1.

In [4]:
print(f'Length of January data columns: {len(df_jan.columns)}')

Length of January data columns: 19


## Q2.

In [7]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime']-df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60
df_jan['duration']                    

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [8]:
std_duration = df_jan['duration'].std()
std_duration

42.59435124195458

## Q3.

In [9]:
def clean_outliers(df):
    mask_1 = df['duration'] >= 1
    mask_60 = df['duration'] <=60

    return df[(mask_1 & mask_60)]


In [10]:
df_jan_range = clean_outliers(df_jan)

df_jan_range['duration'].min(), df_jan_range['duration'].max()

(1.0, 60.0)

In [12]:
fraction = len(df_jan_range)/len(df_jan)
fraction

0.9812202822125979

## Q4.

In [15]:
def preprocess(df):
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)

    feature_list = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
    return feature_list

features_jan = preprocess(df_jan_range)

/tmp/ipykernel_72980/325598935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].astype(str)
/tmp/ipykernel_72980/325598935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].astype(str)


In [16]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=True)
feature_matrix_jan = dv.fit_transform(features_jan)

feature_matrix_jan.shape

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(3009173, 515)

## Q5.

In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_target_jan = df_jan_range['duration']

model = LinearRegression()
model.fit(feature_matrix_jan, y_target_jan)

y_pred_jan = model.predict(feature_matrix_jan)

rmse_jan = np.sqrt(mean_squared_error(y_target_jan, y_pred_jan))

print('RMSE: ', rmse_jan)


RMSE:  7.649261931816197


## Q6.

In [19]:
df_feb['duration'] = (df_feb['tpep_dropoff_datetime']-df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

df_feb_range = clean_outliers(df_feb)

features_feb = preprocess(df_feb_range)
feature_matrix_feb = dv.transform(features_feb)

y_target_feb = df_feb_range['duration']
y_pred_feb = model.predict(feature_matrix_feb)

rmse_feb = np.sqrt(mean_squared_error(y_target_feb, y_pred_feb))

print('RMSE on validation: ', rmse_feb)




/tmp/ipykernel_72980/325598935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].astype(str)
/tmp/ipykernel_72980/325598935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].astype(str)


RMSE on validation:  7.8118186871593
